In [50]:
from config import *
import numpy as np
import pickle

In [51]:
def jaccard(box_a, box_b):
    x_overlap = max(0, min(box_a[2], box_b[2]) - max(box_a[0], box_b[0]))
    y_overlap = max(0, min(box_a[3], box_b[3]) - max(box_a[1], box_b[1]))
    intersection = x_overlap * y_overlap

    area_box_a = (box_a[2] - box_a[0]) * (box_a[3] - box_a[1])
    area_box_b = (box_b[2] - box_b[0]) * (box_b[3] - box_b[1])
    union = area_box_a + area_box_b - intersection

    iou = intersection / union
    return iou

In [52]:
def find_gt_boxes(data_raw, image_file):
    signs_data = data_raw[image_file]
 
    signs_class = []
    signs_box_coords = []  # relative coordinates
    for sign_data in signs_data:
        # Find class label
        sign_class = sign_data['class']
        signs_class.append(sign_class)

        # Calculate relative coordinates
        # (x1, y1, x2, y2), where 1 denotes upper left corner, 2 denotes lower right corner
        abs_box_coords = sign_data["box_coords"]
        scale = np.array([IMG_W, IMG_H, IMG_W, IMG_H])
        box_coords = np.array(abs_box_coords) / scale
        
        signs_box_coords.append(box_coords)

    # Initialize y_true to all 0s (0 -> background)
    y_true_len = 0
    for fm_size in FM_SIZES:
        y_true_len += fm_size[0] * fm_size[1] * NUM_DEFAULT_BOXES
        
    y_true_conf = np.zeros(y_true_len)
    y_true_loc = np.zeros(y_true_len * 4)

    # For each GT box, for each feature map, for each feature map cell, for each default box:
    # 1) Calculate the Jaccard overlap (IOU) and annotate the class label
    # 2) Count how many box matches we got
    # 3) If we got a match, calculate normalized box coordinates and updte y_true_loc
    match_counter = 0
    for i, gt_box_coords in enumerate(signs_box_coords):
        y_true_idx = 0
        #for fm_idx, fm_size in enumerate(FM_SIZES):
        for fm_size in FM_SIZES:
            fm_h, fm_w = fm_size  # feature map height and width
            for row in range(fm_h):
                for col in range(fm_w):
                    for db in DEFAULT_BOXES:
                        # Calculate relative box coordinates for this default box
                        x1_offset, y1_offset, x2_offset, y2_offset = db
                        abs_db_box_coords = np.array([
                            max(0, col + x1_offset),
                            max(0, row + y1_offset),
                            min(fm_w, col+1 + x2_offset),
                            min(fm_h, row+1 + y2_offset)
                        ])
                        scale = np.array([fm_w, fm_h, fm_w, fm_h])
                        db_box_coords = abs_db_box_coords / scale
                        # Calculate Jaccard overlap (i.e. Intersection Over Union, IOU) of GT box and default box
                        iou = jaccard(gt_box_coords, db_box_coords)

                        # If box matches, i.e. IOU threshold met
                        if iou >= IOU_THRESH:
                            # Update y_true_conf to reflect we found a match, and increment match_counter
                            y_true_conf[y_true_idx] = signs_class[i]
                            
                            match_counter += 1

                            # Calculate normalized box coordinates and update y_true_loc
                            abs_box_center = np.array([col + 0.5, row + 0.5])  # absolute coordinates of center of feature map cell
                            abs_gt_box_coords = gt_box_coords * scale  # absolute ground truth box coordinates (in feature map grid)
                            
                            norm_box_coords = abs_gt_box_coords - np.concatenate((abs_box_center, abs_box_center))
                            y_true_loc[y_true_idx*4 : y_true_idx*4 + 4] = norm_box_coords
                            
                            
                            print(y_true_idx*4, y_true_idx*4 + 4, np.sum(y_true_loc), norm_box_coords)
                            
                            
                        y_true_idx += 1
    
    
#     print(y_true_loc)
    return y_true_conf, y_true_loc, match_counter



In [53]:
def encode(data_raw):
    data_prep = {}
    for image_file in data_raw.keys():
        # Find groud-truth boxes based on Jaccard overlap,
        # populate y_true_conf (class labels) and y_true_loc (normalized box coordinates)
        y_true_conf, y_true_loc, match_counter = find_gt_boxes(data_raw, image_file)

        # Only want data points where we have matching default boxes
        if match_counter > 0:
            data_prep[image_file] = {'y_true_conf': y_true_conf, 'y_true_loc': y_true_loc}

    return data_prep


In [54]:
if __name__ == '__main__':
    with open('data_raw_%sx%s.p' % (IMG_W, IMG_H), 'rb') as f:
        data_raw = pickle.load(f)

    data_prep = encode(data_raw)

    with open('data_prep_%sx%s.p' % (IMG_W, IMG_H), 'wb') as f:
        pickle.dump(data_prep, f)


12088 12092 0.9599999999999973 [-1.42 -0.62  1.46  1.54]
33520 33524 -0.27083333333333215 [-0.9375     -1.10666667  0.5         1.27333333]
33528 33532 -0.5416666666666643 [-0.9375     -1.10666667  0.5         1.27333333]
33532 33536 -0.8124999999999964 [-0.9375     -1.10666667  0.5         1.27333333]
33900 33904 -3.0833333333333286 [-0.9375     -2.10666667  0.5         0.27333333]
32656 32660 -0.046666666666666856 [-1.16       -1.14666667  1.14        1.12      ]
32664 32668 -0.09333333333333371 [-1.16       -1.14666667  1.14        1.12      ]
32668 32672 -0.14000000000000057 [-1.16       -1.14666667  1.14        1.12      ]
9376 9380 0.3200000000000003 [-1.38 -0.74  0.78  1.66]
20240 20244 0.5599999999999952 [-0.7  -0.82  1.22  0.86]
20248 20252 1.1199999999999903 [-0.7  -0.82  1.22  0.86]
20252 20256 1.6799999999999855 [-0.7  -0.82  1.22  0.86]
32224 32228 -0.14333333333333265 [-1.3575     -0.82666667  0.8275      1.21333333]
32232 32236 -0.2866666666666653 [-1.3575     -0.8266666

35152 35156 -1.0999999999999988 [-0.95 -1.13  0.67  0.31]
28880 28884 -0.5333333333333332 [-1.255      -0.72333333  0.355       1.09      ]
28888 28892 -1.0666666666666664 [-1.255      -0.72333333  0.355       1.09      ]
35668 35672 0.3566666666666666 [-0.25       -0.25        0.44        0.41666667]
30112 30116 -0.07500000000000018 [-0.975      -1.00333333  0.98        0.92333333]
30120 30124 -0.15000000000000036 [-0.975      -1.00333333  0.98        0.92333333]
30124 30128 -0.22500000000000053 [-0.975      -1.00333333  0.98        0.92333333]
27752 27756 1.3916666666666666 [-1.3825     -0.44333333  2.5275      0.69      ]
27768 27772 0.7833333333333332 [-2.3825     -0.44333333  1.5275      0.69      ]
35872 35876 -0.29000000000000004 [-0.935 -1.15   0.745  1.05 ]
35884 35888 -0.5800000000000001 [-0.935 -1.15   0.745  1.05 ]
31008 31012 -0.06833333333333469 [-1.4875     -0.66        0.8125      1.26666667]
31016 31020 -0.13666666666666938 [-1.4875     -0.66        0.8125      1.26666

14104 14108 2.3999999999999986 [-0.58 -0.86  1.34  1.3 ]
14108 14112 3.599999999999998 [-0.58 -0.86  1.34  1.3 ]
32016 32020 0.7466666666666679 [-0.5475     -0.56333333  0.9475      0.91      ]
25136 25140 0.3200000000000003 [-1.06 -1.54  1.82  1.1 ]
8880 8884 0.0 [-1.58 -0.94  1.3   1.22]
8888 8892 0.0 [-1.58 -0.94  1.3   1.22]
7152 7156 0.8799999999999972 [-0.98 -1.1   1.42  1.54]
7164 7168 1.7599999999999945 [-0.98 -1.1   1.42  1.54]
26928 26932 -0.1200000000000001 [-1.46 -0.82  1.66  0.5 ]
26936 26940 -0.2400000000000002 [-1.46 -0.82  1.66  0.5 ]
6448 6452 1.1199999999999992 [-0.54 -1.06  1.38  1.34]
6460 6464 2.2399999999999984 [-0.54 -1.06  1.38  1.34]
27680 27684 0.7550000000000003 [-1.58  -0.5    1.295  1.54 ]
27688 27692 1.5100000000000007 [-1.58  -0.5    1.295  1.54 ]
28056 28060 0.265000000000001 [-1.58  -1.5    1.295  0.54 ]
33912 33916 1.805000000000001 [-0.02 -0.5   1.48  0.58]
33920 33924 1.345000000000001 [-1.02 -0.5   0.48  0.58]
33928 33932 0.8850000000000007 [-1.02 -

35580 35584 -1.4200000000000017 [-0.73 -1.15  0.41  0.05]
13808 13812 -0.5199999999999996 [-1.54 -1.18  0.5   1.7 ]
27840 27844 0.20000000000000173 [-1.1725 -0.5     1.0125  0.86  ]
27848 27852 0.40000000000000346 [-1.1725 -0.5     1.0125  0.86  ]
27852 27856 0.6000000000000052 [-1.1725 -0.5     1.0125  0.86  ]
35700 35704 0.19000000000000072 [-0.345 -0.35   0.435  0.45 ]
35904 35908 -1.4466666666666672 [-1.21       -0.95        0.23        0.48333333]
35912 35916 -2.8933333333333344 [-1.21       -0.95        0.23        0.48333333]
12112 12116 -0.16000000000000547 [-1.38 -1.22  1.26  1.18]
12120 12124 -0.32000000000001094 [-1.38 -1.22  1.26  1.18]
12124 12128 -0.4800000000000164 [-1.38 -1.22  1.26  1.18]
32272 32276 -0.5 [-1.1375 -0.94    0.8175  0.76  ]
32280 32284 -1.0 [-1.1375 -0.94    0.8175  0.76  ]
32284 32288 -1.5 [-1.1375 -0.94    0.8175  0.76  ]
29520 29524 0.40000000000000013 [-0.7175     -1.02333333  1.2375      0.90333333]
29528 29532 0.8000000000000003 [-0.7175     -1.023

27660 27664 -1.2774999999999994 [-0.5    -0.5     0.7075  0.52  ]
34176 34180 0.8800000000000003 [-0.64 -0.96  1.34  1.14]
34184 34188 1.7600000000000007 [-0.64 -0.96  1.34  1.14]
34188 34192 2.640000000000001 [-0.64 -0.96  1.34  1.14]
34000 34004 -0.08000000000000007 [-1.31 -0.5   0.73  1.  ]
34008 34012 -0.16000000000000014 [-1.31 -0.5   0.73  1.  ]
34012 34016 -0.2400000000000002 [-1.31 -0.5   0.73  1.  ]
32368 32372 -0.19333333333333513 [-0.64       -0.88333333  0.74        0.59      ]
32380 32384 -0.38666666666667027 [-0.64       -0.88333333  0.74        0.59      ]
32320 32324 0.394999999999996 [-0.8025 -0.77    1.0375  0.93  ]
32328 32332 0.789999999999992 [-0.8025 -0.77    1.0375  0.93  ]
32332 32336 1.184999999999988 [-0.8025 -0.77    1.0375  0.93  ]
31744 31748 -0.0983333333333345 [-1.0275     -0.62        0.5825      0.96666667]
31752 31756 -0.196666666666669 [-1.0275     -0.62        0.5825      0.96666667]
34352 34356 -0.37999999999999945 [-0.72 -1.33  0.9   0.77]
34364 34

...

28832 28836 0.1900000000000004 [-0.8425     -1.51666667  1.5725      0.97666667]
14160 14164 -0.3200000000000003 [-1.34 -0.98  0.82  1.18]
14168 14172 -0.6400000000000006 [-1.34 -0.98  0.82  1.18]
14172 14176 -0.9600000000000009 [-1.34 -0.98  0.82  1.18]
33792 33796 -1.2683333333333362 [-1.0325     -1.14333333  0.5775      0.33      ]
33804 33808 -2.5366666666666724 [-1.0325     -1.14333333  0.5775      0.33      ]
35572 35576 -2.516666666666673 [-0.43 -0.37  0.41  0.41]
34400 34404 -0.3800000000000001 [-0.63 -1.15  0.99  0.41]
34412 34416 -0.7600000000000002 [-0.63 -1.15  0.99  0.41]
27904 27908 0.8183333333333307 [-1.0325     -0.5         1.0375      1.31333333]
27912 27916 1.6366666666666614 [-1.0325     -0.5         1.0375      1.31333333]
27916 27920 2.454999999999992 [-1.0325     -0.5         1.0375      1.31333333]
28272 28276 1.2733333333333228 [-1.0325     -1.5         1.0375      0.31333333]
29968 29972 0.8916666666666675 [-0.8875     -0.43666667  0.9525      1.26333333]
2998

34564 34568 0.30000000000000027 [-0.25 -0.38  0.41  0.28]
22288 22292 0.52 [-1.5  -0.82  1.02  1.82]
23056 23060 -0.96 [-1.5  -1.82  1.02  0.82]
32800 32804 -1.4191666666666678 [-0.5        -0.95666667  0.7075      0.29      ]
32800 32804 -0.5166666666666659 [-0.5        -1.12666667  0.65        0.46      ]
32812 32816 -1.0333333333333319 [-0.5        -1.12666667  0.65        0.46      ]
28432 28436 0.3533333333333337 [-0.625      -1.14        1.445       0.67333333]
28440 28444 0.7066666666666674 [-0.625      -1.14        1.445       0.67333333]
9616 9620 0.3200000000000003 [-1.14 -1.34  1.98  0.82]
9632 9636 -1.3599999999999994 [-2.14 -1.34  0.98  0.82]
7312 7316 0.32000000000000384 [-0.54 -1.34  1.38  0.82]
7324 7328 0.6400000000000077 [-0.54 -1.34  1.38  0.82]
34320 34324 -0.5800000000000005 [-1.24 -1.15  0.92  0.89]
34328 34332 -1.160000000000001 [-1.24 -1.15  0.92  0.89]
34332 34336 -1.7400000000000015 [-1.24 -1.15  0.92  0.89]
20544 20548 1.4399999999999977 [-0.98 -0.94  1.42  1

32512 32516 0.6883333333333317 [-1.36       -0.52333333  1.055       1.51666667]
32520 32524 1.3766666666666634 [-1.36       -0.52333333  1.055       1.51666667]
32880 32884 0.06499999999999506 [-1.36       -1.52333333  1.055       0.51666667]
32888 32892 -1.2466666666666733 [-1.36       -1.52333333  1.055       0.51666667]
30720 30724 -0.44166666666666643 [-1.025      -1.07666667  0.47        1.19      ]
30732 30736 -0.8833333333333329 [-1.025      -1.07666667  0.47        1.19      ]
27776 27780 0.7183333333333336 [-0.7375     -0.21666667  0.6425      1.03      ]
27788 27792 1.4366666666666672 [-0.7375     -0.21666667  0.6425      1.03      ]
33972 33976 1.2166666666666683 [-0.45 -0.35  0.27  0.31]
32624 32628 -0.6450000000000014 [-1.115 -0.92   0.61   0.78 ]
32632 32636 -1.2900000000000027 [-1.115 -0.92   0.61   0.78 ]
32636 32640 -1.935000000000004 [-1.115 -0.92   0.61   0.78 ]
31664 31668 1.0066666666666713 [-0.5125     -0.58333333  1.2125      0.89      ]
31672 31676 2.0133333333

31424 31428 -0.1616666666666653 [-1.21       -0.58333333  0.515       1.11666667]
31432 31436 -0.3233333333333306 [-1.21       -0.58333333  0.515       1.11666667]
29680 29684 0.40999999999999925 [-0.54 -0.91  1.07  0.79]
29688 29692 0.8199999999999985 [-0.54 -0.91  1.07  0.79]
10240 10244 1.7763568394002505e-15 [-1.02 -1.02  1.14  0.9 ]
10248 10252 3.552713678800501e-15 [-1.02 -1.02  1.14  0.9 ]
10252 10256 5.329070518200751e-15 [-1.02 -1.02  1.14  0.9 ]
27904 27908 0.4683333333333346 [-1.665      -0.27333333  0.98        1.42666667]
34464 34468 0.020000000000001794 [-0.88 -1.24  0.5   1.64]
34476 34480 0.04000000000000359 [-0.88 -1.24  0.5   1.64]
30696 30700 -0.8066666666666684 [-1.0375     -0.79333333  0.4575      0.56666667]
34304 34308 0.5200000000000002 [-0.72 -0.52  0.78  0.98]
34316 34320 1.0400000000000005 [-0.72 -0.52  0.78  0.98]
368 372 0.9600000000000026 [-1.42 -0.38  0.98  1.78]
1136 1140 -0.07999999999999474 [-1.42 -1.38  0.98  0.78]
1144 1148 -1.1199999999999921 [-1.42

14840 14844 1.119999999999994 [-0.74 -1.02  1.42  0.9 ]
14844 14848 1.6799999999999908 [-0.74 -1.02  1.42  0.9 ]
30576 30580 -1.674999999999999 [-1.455      -1.83333333  0.5         1.11333333]
34668 34672 -1.134999999999998 [-0.52 -0.5   0.5   1.06]
